In [2]:
import pandas as pd
from glob import glob

In [2]:
for folder in glob("../data/labels/*/"):
    with open(folder + "synsets.txt") as f:
        labels = list(map(lambda x: int(x[:-1]) if x[:-1].isdigit() else x[:-1], f.readlines()))
    columns = ["folder", "name", "count"] + list(filter(lambda x: isinstance(x, str), labels))
    df = pd.DataFrame(columns=columns)
    folder_name = folder[folder.find("/") + 1: -1]
    with open(folder + "default.txt") as f:
        for line in f:
            line = line.replace('"', "")
            line = line.replace("\n", "")
            image = {"folder": folder_name, "name": line[: line.find(".jpg") + 4]}
            tags = list(map(lambda x: int(x), line[line.find(".jpg") + 5:].split(" ")))
            for tag in tags:
                if isinstance(labels[tag], int):
                    image["count"] = labels[tag]
                else:
                    image[labels[tag]] = True
            df = pd.concat([df, pd.DataFrame.from_records([image])], ignore_index=True)
    df = df.fillna(False)
    df.to_csv(folder + "/" + "labels.csv", index=False)

In [ ]:
# create train & test folders with images and labels
import os
from PIL import Image

set_name = "train"
data = 'ESOS'
path_folder = f"/cvhci/temp/p22g5/data/"
df = pd.read_csv(f"../data/{data}/{set_name}.csv")
for _, row in df.iterrows():
    img_PIL = Image.open(path_folder + row['name'])
    path_image = f"{path_folder}{data}/{set_name}/{row['count']}/"
    if not os.path.exists(path_image):
        os.makedirs(path_image)
    path_image += row['name'][row['name'].rfind('/') + 1:]
    if img_PIL.mode in ("RGBA", "P"):
        img_PIL = img_PIL.convert('RGB')
    img_PIL.save(path_image)